<a href="https://colab.research.google.com/github/Trickshotblaster/nn-practices/blob/main/MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests

url = 'https://raw.githubusercontent.com/karpathy/makemore/master/names.txt'

response = requests.get(url)

with open('names.txt', 'wb') as f:
  f.write(response.content)

In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [4]:
len(words)

32033

In [6]:
# stoi and itos
chars = sorted(list(set(''.join(words))))
stoi = {char:i+1 for i, char in enumerate(chars)}
stoi['.'] = 0
itos = {i:char for char, i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [9]:
# make dataset
block_size = 3 # context length
X, Y = [], []
for w in words[:5]:
  print(w)
  context = [0] * block_size # list of zeros of length block size
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    print(''.join(itos[i] for i in context), '---->', itos[ix])
    context = context[1:] + [ix] # crop to remove previous, append new to end

X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... ----> e
..e ----> m
.em ----> m
emm ----> a
mma ----> .
olivia
... ----> o
..o ----> l
.ol ----> i
oli ----> v
liv ----> i
ivi ----> a
via ----> .
ava
... ----> a
..a ----> v
.av ----> a
ava ----> .
isabella
... ----> i
..i ----> s
.is ----> a
isa ----> b
sab ----> e
abe ----> l
bel ----> l
ell ----> a
lla ----> .
sophia
... ----> s
..s ----> o
.so ----> p
sop ----> h
oph ----> i
phi ----> a
hia ----> .


In [10]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [13]:
# C should be a 27 x embedding dimension matrix
C = torch.randn((27, 2))

In [14]:
C[stoi['a']]

tensor([0.4833, 1.6870])

In [17]:
aonehot = F.one_hot(torch.tensor(stoi['a']), num_classes=27).float()
aonehot @ C

tensor([0.4833, 1.6870])

In [18]:
C[[5, 6, 7]]

tensor([[ 1.9274,  0.6570],
        [ 0.9623,  2.2364],
        [ 0.3431, -0.2882]])

In [20]:
C[torch.tensor([0,0,0])]

tensor([[-0.5262, -0.4338],
        [-0.5262, -0.4338],
        [-0.5262, -0.4338]])

In [22]:
for data in X[:5]:
  enc = C[data]
  print(enc)

tensor([[-0.5262, -0.4338],
        [-0.5262, -0.4338],
        [-0.5262, -0.4338]])
tensor([[-0.5262, -0.4338],
        [-0.5262, -0.4338],
        [ 1.9274,  0.6570]])
tensor([[-0.5262, -0.4338],
        [ 1.9274,  0.6570],
        [ 2.3748,  0.4758]])
tensor([[1.9274, 0.6570],
        [2.3748, 0.4758],
        [2.3748, 0.4758]])
tensor([[2.3748, 0.4758],
        [2.3748, 0.4758],
        [0.4833, 1.6870]])


In [25]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [106]:
w1 = torch.randn(6, 100)
b1 = torch.randn(100)
w2 = torch.randn(100, 27)
b2 = torch.randn(27)

wi1 = emb.view(-1, 6) @ w1
wi1b = wi1 + b1
wi1ba = wi1b.tanh()
wi2 = wi1ba @ w2
logits = wi2 + b2
counts = logits.exp()
probs = counts / counts.sum(1, keepdim=True)
probs.shape

torch.Size([32, 27])

In [52]:
%time
torch.cat([emb[:, 0, :], emb[:, 1, :], emb[:, 2, :]], 1).shape

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 6.68 µs


torch.Size([32, 6])

In [93]:
%time
torch.flatten(emb, start_dim=1).shape

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.87 µs


torch.Size([32, 6])

In [75]:
%time
torch.cat(torch.unbind(emb, 1), 1).shape

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.15 µs


torch.Size([32, 6])

Idk bruh I think mine is still better

In [94]:
a = torch.arange(18)
a, a.shape

(tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17]),
 torch.Size([18]))

In [95]:
a.view(3, 6) # holy moly where has this been all my life

tensor([[ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11],
        [12, 13, 14, 15, 16, 17]])

In [97]:
%time
emb.view(32, 6).shape

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.77 µs


torch.Size([32, 6])

2 microseconds bruv